In [1]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

In [2]:
def hod_rd_sub1(mean):
    for i in range(my,shape[1]-lmd,lmd-2):
        ver_rd(i,mean)

In [3]:
def hod_rd_sub2(mean1):
    for i in range(my+lmd,0, -(lmd-2)):
        ver_rd(i,mean1)

In [70]:
def hor_rd(mean):
    hod_rd_sub1(mean)
    hod_rd_sub2(mean)

In [78]:
def ver_rd(x,mean):
    lm1 = lim1
    lm2 = lim2
    for i in range(mx,lmd+1,-(lmd-10)):
        lm1 += lm1*0.03
        lm2 += lm2*0.03
        low = mean - [0,lm1,lm2]
        high = mean + [180,lm1,lm2]
        newscr[i-lmd:i, x:x+lmd] = cv2.inRange(hls[i-lmd:i, x:x+lmd],low,high)
        res = [0]+[np.mean(hls[i-lmd:i, x:x+lmd,j]) for j in range(1,3)]
        res = np.array(res)
        if(abs(res[1]-mean[1])<=15 and abs(res[2]-mean[2])<=15):
            mean = (4*res+6*mean)/10

In [85]:
import numpy as np
import pandas as pd
import time
import cv2
import threading
# lim = 4*std
lmd = 40

kernel = np.ones((3,3),np.uint8)
vrec = cv2.VideoCapture('v4.mp4',0)
shape = [480,680]#scr.shape

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*8#*6
mask_prev1 = np.zeros((shape[0],shape[1]),np.uint8)
mask_prev2 = np.zeros((shape[0],shape[1]),np.uint8)
mean_prev = 0
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

while True:
    start_time = time.time()
    ret, scr = vrec.read()
    ret, scr = vrec.read()
#     ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
    
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr = cv2.resize(scr, (680,480)) 
    
    newscr = np.zeros((shape[0],shape[1]),np.uint8)
    scr = cv2.bilateralFilter(scr,7,12,12)
    hls = cv2.cvtColor(scr, cv2.COLOR_BGR2HLS)
    mean = [0]+[np.mean(hls[440:480,200:480, i]) for i in range(1,3)]
    std = [np.std(hls[440:480,200:480, i]) for i in range(1,3)]
    lim1 = (lim1 + 16*std[0])/5#(1:4(4))/5
    lim2 = (lim2 + 16*std[1])/5
#     print(lim1,lim2)
    mean = np.array(mean)
    mean = 0.8*mean + mean_prev*0.2
    mean_prev = mean
#     print(mean)
    hor_rd(mean)
    band =  cv2.bitwise_and(scr,scr,mask = newscr)
    
    gray = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
    mean1 = np.mean(gray[440:480,200:480])
    mean1 = np.array(mean1)
    low1 = mean1 - 25
    high1 = mean1 + 25
    
    mask1 = cv2.inRange(gray,low1,high1)
    band1 =  cv2.bitwise_and(scr,scr,mask = mask1)
    
#     newscr = cv2.morphologyEx(newscr,cv2.MORPH_OPEN,kernel, iterations = 1)
#     newscr = cv2.morphologyEx(newscr,cv2.MORPH_CLOSE,kernel, iterations = 1)

    
        
    mask3 = newscr+mask1
#     band3 =  cv2.bitwise_and(scr,scr,mask = mask3)
    
    mask4 = mask_prev1 + mask3 #+ mask_prev2
#     mask_pres2 = mask_prev1
    mask_prev1 = mask3 
    
#     canny = cv2.Canny(gray,10,50)
#     cv2.imshow("canny", canny)

#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)
    
    
#     cv2.imshow('blur',band)
#     cv2.imshow('mean1',band1)
#     cv2.imshow('band3',band3)
    cv2.imshow('newscr',newscr)
    cv2.imshow('mean1',mask1)
    cv2.imshow('band3',mask4)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_OPEN,kernel, iterations = 1)
#     mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)


    dist_transform = cv2.distanceTransform(mask4,cv2.DIST_L2,5)
    ret, mask4 = cv2.threshold(dist_transform,0.07*dist_transform.max(),255,0)
    mask4 = cv2.morphologyEx(mask4,cv2.MORPH_CLOSE,kernel, iterations = 1)
    ret, mask4 = cv2.threshold(dist_transform,0.07*dist_transform.max(),255,0)
    cv2.imshow('band4',mask4)
    mask4 = np.uint8(mask4)
    im2, contours, hierarchy = cv2.findContours(mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#     cnt = contours[0]
    scr1 = cv2.drawContours(scr, contours, -1, (0,255,0), 3)
    
    
#     n_mask4 = cv2.bitwise_not(mask4)
    
#     im2, contours, hierarchy = cv2.findContours(n_mask4, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# #     cnt = contours[0]
#     scr2 = cv2.drawContours(scr, contours, -1, (0,255,0), 3)
    
    
    cv2.imshow('scr1',scr1)
#     cv2.imshow('scr2',scr2)
#     print("FPS: ", 1.0 / (time.time() - start_time))
#     time.sleep(0.2)
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
[   0.          120.25528571   21.75521429]
FPS:  23.93025697202063
[   0.          143.64305714   25.78482857]
FPS:  24.87990414160468
[   0.          148.53739714   26.91668   ]
FPS:  23.584837971423593
[   0.          150.51105086   27.46955029]
FPS:  25.405858551595475
[   0.          150.44063874   27.58412434]
FPS:  24.393146685587336
[   0.          149.97298489   27.72111058]
FPS:  25.051987767584098
[   0.          151.03545412   29.70757926]
FPS:  25.186476911067075
[   0.          154.18980511   33.91251585]
FPS:  23.656803799253236
[   0.          155.00946102   36.65228888]
FPS:  24.876215103762004
[   0.          155.30203506   37.99688635]
FPS:  24.719343222711387
[   0.          154.7921213    39.58759156]
FPS:  25.15339822127869
[   0.          155.44192426   40.4937326 ]
FPS:  25.46014325603982
[   0.          154.20288485   39.85496081]
FPS:  25.615321664569873
[   0.          152.98643411   40.24206359]
FPS:  25.48226273868904
[   0.          151.51935825   

[  0.          74.03369185  53.96493968]
FPS:  22.895922266499262
[  0.          75.15652408  54.01513079]
FPS:  22.55390174653704
[  0.          77.95966196  52.60445473]
FPS:  22.933102961841076
[  0.          79.22321811  52.26196237]
FPS:  20.781782327351284
[  0.          79.63507219  51.97910676]
FPS:  22.85724250681199
[  0.          79.99530015  51.68424992]
FPS:  23.106439475322414
[  0.          82.60463146  50.37356427]
FPS:  19.396701782295434
[  0.          84.20835486  49.79071285]
FPS:  23.178584849355644
[  0.          85.12702812  49.39514257]
FPS:  23.154017709276392
[  0.          85.98583419  48.97845709]
FPS:  21.525257242564983
[  0.          86.87702398  48.56711999]
FPS:  22.601908682836402
[  0.          87.0164048   48.48506685]
FPS:  20.187342673834884
[  0.          85.61249524  48.96115623]
FPS:  23.158364345108605
[  0.          84.83692762  49.47680267]
FPS:  21.104584404671453
[  0.          85.33309981  48.90421768]
FPS:  21.941326637371834
[  0.       

[   0.          132.47656562   33.44462439]
FPS:  23.21039466985413
[   0.          129.59345598   33.31085345]
FPS:  21.663450612565338
[   0.          128.17976262   33.62888498]
FPS:  21.05088182447828
[   0.          126.6953811    33.64956271]
FPS:  22.186802084159858
[   0.          125.17907622   34.34998397]
FPS:  23.17179808738792
[   0.          124.84710096   35.28035394]
FPS:  21.654279430441832
[   0.          121.97234876   35.40578507]
FPS:  21.229565367036326
[   0.          119.04754118   36.09144273]
FPS:  23.597841803523103
[   0.          113.75286538   38.05164569]
FPS:  22.543718960290672
[   0.          111.51471593   38.93304342]
FPS:  20.823361780928693
[   0.          116.1227289    37.73425154]
FPS:  22.540326741186586
[   0.          120.72333149   37.61892174]
FPS:  23.637472315053284
[   0.          115.68759487   37.14221292]
FPS:  23.598372876778612
[   0.          101.78866183   40.82194258]
FPS:  23.96224819753425
[  0.          96.92194665  42.2540313

FPS:  21.708861480174114
[  0.          72.84291798  54.61117905]
FPS:  24.151694354072497
[  0.          72.39936931  54.70923581]
FPS:  21.447877356078504
[  0.          71.14030243  55.53027573]
FPS:  23.330073812026853
[  0.          70.00727477  56.32426943]
FPS:  23.856756080358565
[  0.          69.53566924  56.93392532]
FPS:  23.55490410805043
[  0.          67.99541956  58.31342792]
FPS:  21.05405188338286
[  0.          68.18251248  59.11518558]
FPS:  23.38131525695842
[  0.          71.19157393  56.34003712]
FPS:  23.678572839924353
[  0.          74.01267193  54.74557885]
FPS:  21.196306833974297
[  0.          76.60253439  53.0606872 ]
FPS:  21.520618583149048
[  0.          79.00100688  51.7392803 ]
FPS:  22.24953318621626
[  0.          81.66612995  49.93528463]
FPS:  24.261220145649318
[  0.          82.99494028  48.90827121]
FPS:  23.592266977157546
[  0.          89.67863091  43.68093996]
FPS:  20.195507619712547
[  0.          99.43786904  37.21747371]
FPS:  22.09260

FPS:  23.19268327748471
[  0.          57.0004583   44.13380575]
FPS:  25.28837144803719
[  0.          62.67452023  44.40754686]
FPS:  24.219052788396024
[  0.          69.64311833  44.06579509]
FPS:  24.48327914402293
[  0.          71.83855224  43.88901616]
FPS:  24.211782906357875
[  0.          67.00806759  44.49601752]
FPS:  24.043704319412996
[  0.          57.1458278   45.46384636]
FPS:  24.023460410557185
[  0.          54.76966556  47.79941213]
FPS:  24.656567259815766
[  0.          57.13750454  45.71545385]
FPS:  24.45230571911619
[  0.          59.50364377  48.04901934]
FPS:  24.65178497960527
[  0.          62.02994304  51.55251815]
FPS:  24.075999793354036
[  0.          71.67577432  53.36214649]
FPS:  24.573072348802192
[  0.          84.23051201  47.90835787]
FPS:  24.112540026559813
[   0.         101.2941024   42.412743 ]
FPS:  23.08088178646504
[   0.          114.76010619   39.1525486 ]
FPS:  23.566550734083616
[   0.          124.95980695   38.57336686]
FPS:  23.3

In [78]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

vrec = cv2.VideoCapture('v2.mp4',0)
shape = [480,680]

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*6*6
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

ret, scr1 = vrec.read()
scr1 = cv2.resize(scr1, (680,480)) 
scr1 = cv2.cvtColor(scr1, cv2.COLOR_BGR2GRAY)
ret, scr2 = vrec.read()
scr2 = cv2.resize(scr2, (680,480)) 
scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
while True:
    start_time = time.time()
    scr = scr1
    scr1 = scr2
    ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr2 = cv2.resize(scr2, (680,480)) 
    scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
    
    d0 = abs(scr1-scr)
    d1 = abs(scr2-scr)
    d2=d0+d1
    
    cv2.imshow('d0',d0)
    cv2.imshow('d1',d1)
    cv2.imshow('d2',d2)
#     cv2.imshow('d3',d3)
#     cv2.imshow('d4',d4)

    cv2.imshow('scr1',scr)
    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
FPS:  6.805267291005497
FPS:  237.974695035461
FPS:  213.0818939239992
FPS:  233.77014825548991
FPS:  230.83676389653274
FPS:  211.23609991941981
FPS:  220.8458298230834
FPS:  214.39983642590605
FPS:  202.9174649250121
FPS:  214.3669630992538
FPS:  199.08410860072146
FPS:  201.45552353506244
FPS:  143.248087431694
FPS:  175.0471182337966
FPS:  165.47536197577622
FPS:  182.2421898761677
FPS:  90.95708368572853
FPS:  185.3590242177833
FPS:  185.9754356404913
FPS:  168.23649271990695
FPS:  185.3180753766624
FPS:  205.09041122683487
FPS:  223.36265843007774
FPS:  225.17335051269663
FPS:  155.42518342844437
FPS:  178.0945182794786
FPS:  232.61627197604128
FPS:  183.5260348297891
FPS:  243.2608746085141
FPS:  241.13510405887087
FPS:  225.84018953262978
FPS:  171.67958740943882
FPS:  217.8180307436643
FPS:  209.98818464003205
FPS:  222.87602954460917
FPS:  169.02974127508665
FPS:  147.40648063541155
FPS:  184.47853624208304
FPS:  212.76842692639374
FPS:  215.47927048548678
FPS:  157.6